In [ ]:
import os
from math import pi
import numpy as np
import random
import itertools
import scipy.optimize as opt
import matplotlib.pyplot as plt
import qiskit
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.quantum_info import Statevector
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import SamplerV2 as Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
# from qiskit_aer.noise import NoiseModel
# from qiskit_ibm_provider import IBMProvider


In [ ]:
shots = 100
epsilon_loss = 1e-3 # loss added if test_counts entry is not found
epsilon_parameters=0.1 # scaling factor for initial random parameters (chosen between 0 to 2pi otherwise)

n_qubits = 3
n_bases = 27
depth = 10

max_iter = 1000

param_name = "theta"

# save checkpoints in case the connection to qpu breaks
# useful for running on real qpu, but not necessary for local simulation
have_checkpoints = False
checkpoint_path = "Alt_ideal_ghz_q3_b27_d6_sh100_checkpoints"

#seed = 1000

In [ ]:
#random.seed(seed)
#rng = np.random.default_rng(seed)

In [ ]:
# if have_checkpoints:
#     os.mkdir(checkpoint_path)

In [ ]:
# backend = AerSimulator()
# sampler = Sampler.from_backend(backend)
# pm = generate_preset_pass_manager(backend=backend)

simulator_ideal = AerSimulator()
#simulator_ideal.set_options(seed_simulator=seed)
# sampler_ideal = Sampler.from_backend(simulator_ideal)
pm_ideal = generate_preset_pass_manager(backend=simulator_ideal, optimization_level=1)

# sampler = sampler_ideal
pm = pm_ideal

In [ ]:
def create_ghz_state(n):
    qc = QuantumCircuit(n)
    qc.h(0)
    for i in range(n - 1):
        qc.cx(i, i + 1)
    return qc

def get_circ_for_basis(qc: QuantumCircuit, basis: str):
    circ = qc.copy()
    for i, s in enumerate(basis[::-1]): # here, I am using the same endian convention as qiskit, where the last bit represent the first qubit
        if s == 'Z':
            pass
        elif s == 'X':
            circ.ry(-pi/2, i)
        elif s == 'Y':
            circ.rx(pi/2, i)
        else:
            raise ValueError(f"{s} is invalid basis")

    circ.measure_all()

    return circ

def get_isa_circ_list_for_bases(qc: QuantumCircuit, basis_list, pass_manager):
    circ_list = [get_circ_for_basis(qc, basis) for basis in basis_list]
    isa_circ_list = pass_manager.run(circ_list)
    return isa_circ_list


def measure_isa_circ_list_fs(isa_circ_list, basis_list, shots=shots):
    #sampler = Sampler(seed =rng.integers(1000000) )
    sampler = Sampler()
    result_list = sampler.run(isa_circ_list, shots=shots).result()
    counts_list = [result.data.meas.get_counts() for result in result_list]
    results_dic = {basis:counts for (basis, counts) in zip(basis_list, counts_list)}
    return results_dic

# randomly chooses n_bases from possible basis (no duplicates)
def get_basis_list(n_bases,n_qubits):
    bases = ["X", "Y", "Z"]
    basis_list = ["".join(p) for p in itertools.product(bases, repeat=n_qubits)]
    culled_basis_list = random.sample(basis_list, n_bases)
    return culled_basis_list




# def measure_circ_for_all_basis(qc, basis_list, shots=total_shots, sampler=None):
#     results_dic = {}
#     qc_list = [get_circ_for_basis(qc, basis, simulator) for basis in basis_list]
#     job = simulator.run(qc_list, shots=shots)
#     result = job.result()
#     counts_list = result.get_counts()
#     results_dic = {basis:counts for (basis, counts) in zip(basis_list, counts_list)}
    
#     return results_dic

In [ ]:
basis_list = get_basis_list(n_bases, n_qubits)
ghz_circ = create_ghz_state(n_qubits)
ghz_circ_isa_list = get_isa_circ_list_for_bases(ghz_circ, basis_list, pm_ideal)
GHZ_measurement = measure_isa_circ_list_fs(ghz_circ_isa_list, basis_list, shots=shots)

In [ ]:
print(basis_list)

In [ ]:
print(GHZ_measurement)

In [ ]:
def initialize_theta_random(circ_depth=10, num_qbits=3):
    """
    Initialize the theta parameter vector
    :param circ_depth: int, number of parameterized layers in circuit
    :param num_qbits: int, number of qbits
    :return: np.array, values of theta
    """

    #theta = rng.random(size=(circ_depth, num_qbits))
    theta = np.random.rand(circ_depth, num_qbits)
    theta = 2*pi*epsilon_parameters*theta
    return theta

In [ ]:
print(initialize_theta_random())

In [ ]:
def construct_variational_circ_ansatz(num_qbits, circ_depth, param_name = param_name):
    """
    Generate a parameterized variational quantum circuit
    
    """

    num_parameters = num_qbits * circ_depth
    theta_vec = ParameterVector(param_name, length=num_parameters)
    var_circ = QuantumCircuit(num_qbits)


    for layer in range(circ_depth - 1):
        # Compute if layer is odd or even to apply rx or ry gate to circuit
        is_odd_step = (layer + 1) % 2

        for qbit in range(num_qbits):
            if is_odd_step:
                var_circ.rx(theta_vec[layer * num_qbits + qbit], qbit)
            else:
                var_circ.ry(theta_vec[layer * num_qbits + qbit], qbit)

        # Apply CX gates
        for qbit in range((1-is_odd_step), num_qbits-1, 2):
            # isOddStep may subtract 1 if True, to correctly apply cx gate location
            var_circ.cx(qbit , qbit + 1)
            var_circ.barrier() # for visualization only

    for qbit in range(num_qbits):  # bonus layer at the end only has rx gates and no cx
        var_circ.rx(theta_vec[(circ_depth - 1) * num_qbits + qbit], qbit)

    return var_circ, theta_vec


In [ ]:
ansatz, param_vec = construct_variational_circ_ansatz(n_qubits, depth, param_name=param_name)

In [ ]:
ansatz.draw(output="mpl", style="clifford")

In [ ]:
ansatz_isa_list = get_isa_circ_list_for_bases(ansatz, basis_list, pm)

In [ ]:
def get_ansatz_output(ansatz_isa_list, parameters, basis_list, shots, param_placeholder=param_vec):
    circ_run_list = [circ.assign_parameters({param_placeholder:parameters}) for circ in ansatz_isa_list]
    return measure_isa_circ_list_fs(isa_circ_list=circ_run_list, basis_list=basis_list, shots=shots)

In [ ]:
def KL_divergence(true_data, test_data, epsilon_loss = epsilon_loss):
    loss = 0

    # here, let me assume true data and test data can have different number of shots
    # but the shot in each basis should be the same
    true_data_total_shots = sum(list(true_data.values())[0].values())
    test_data_total_shots = sum(list(test_data.values())[0].values())

    for basis in true_data: # just a note that if there are hallucinated measurements in 
        # bases/states not included in the true data then the kl divergence will not account for these (at least as presented in the paper)
        for state in true_data[basis]:
            true_counts = true_data[basis][state]
            test_counts = test_data[basis].get(state, 0)
            true_prob   = true_counts / true_data_total_shots
            test_prob   = test_counts / test_data_total_shots
            loss += true_prob *(np.log((true_prob)/(test_prob +epsilon_loss)))

    return loss / len(true_data)

In [ ]:
global loss_values
global thetas

loss_values = []
thetas = []

In [ ]:
def compute_kl_loss(theta_vector, ansatz_isa_list,  basis_list, true_data, shots, param_placeholder=param_vec, \
        print_message=True, have_checkpoints=have_checkpoints, checkpoint_path=checkpoint_path):
    # theta = np.reshape(theta_vector, (circ_depth, num_qbits))
    # tempcirc = construct_variational_circ(theta=theta)
    test_data = get_ansatz_output(ansatz_isa_list=ansatz_isa_list, parameters=theta_vector, basis_list=basis_list, \
        shots=shots, param_placeholder=param_placeholder)
    # test_data = measure_circ_for_all_basis(qc = tempcirc, basis_list=basis_list, simulator=simulator)
    loss = KL_divergence(true_data, test_data) + \
        KL_divergence(test_data, true_data) # symmetrizing the loss

    global loss_values
    global thetas 
    loss_values.append(loss)
    thetas.append(theta_vector)

    num_iter = len(loss_values)

    if print_message:
        print(f"Iteration {num_iter:5}; current loss is {loss}")

    if have_checkpoints:
        # save checkpoints in case the connection to qpu breaks
        # useful for running on real qpu, but not necessary for local simulation
        np.save(os.path.join(checkpoint_path, f"theta_iter{num_iter}.npy"), thetas)
    
    return loss


In [ ]:
def compute_fidelity(psi, phi):
    """
    Compute the fidelity (a measure of similarity) between the two states
    :param psi: qiskit.Statevector, our target state |psi>
    :param phi: qiskit.Statevector, our estimated state |phi>
    :return: float, fidelity
    """

    fidelity = qiskit.quantum_info.state_fidelity(psi, phi)
    print(f"psi: {psi}, phi: {phi}")
    # fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    # plot_bloch_multivector(psi)
    # axes[0].set_title("psi")
    # plot_bloch_multivector(phi)
    # axes[1].set_title("phi")
    return fidelity



ghz_state = Statevector.from_instruction(ghz_circ)
#ansatz_state = Statevector.from_instruction(ansatz.assign_parameters({param_vec : results.x}))

#final_fidelity = qiskit.quantum_info.state_fidelity(ghz_state, ansatz_state)

In [ ]:
def compute_kl_loss_filled(theta_vector):
    return compute_kl_loss(theta_vector, ansatz_isa_list, basis_list, GHZ_measurement, shots, param_vec, print_message=True)

def compute_kl_loss_filled_quiet(theta_vector):
    return compute_kl_loss(theta_vector, ansatz_isa_list, basis_list, GHZ_measurement, shots, param_vec, print_message=False)


In [ ]:
def mod_compute_kl_loss(theta_vector, ansatz_isa_list,  basis_list, true_data, shots, param_placeholder=param_vec, \
        print_message=True, have_checkpoints=have_checkpoints, checkpoint_path=checkpoint_path):
    # theta = np.reshape(theta_vector, (circ_depth, num_qbits))
    # tempcirc = construct_variational_circ(theta=theta)
    test_data = get_ansatz_output(ansatz_isa_list=ansatz_isa_list, parameters=theta_vector, basis_list=basis_list, \
        shots=shots, param_placeholder=param_placeholder)
    # test_data = measure_circ_for_all_basis(qc = tempcirc, basis_list=basis_list, simulator=simulator)
    loss = KL_divergence(true_data, test_data) + \
        KL_divergence(test_data, true_data) # symmetrizing the loss

    global loss_values
    global thetas 
    loss_values.append(loss)
    thetas.append(theta_vector)

    num_iter = len(loss_values)

    if print_message:
        print(f"Iteration {num_iter:5}; current loss is {loss}")

    if have_checkpoints:
        # save checkpoints in case the connection to qpu breaks
        # useful for running on real qpu, but not necessary for local simulation
        np.save(os.path.join(checkpoint_path, f"theta_iter{num_iter}.npy"), thetas)
    
    return (loss, test_data)

In [ ]:
from qiskit_algorithms.gradients import ParamShiftSamplerGradient
from qiskit.primitives import Sampler as qiskitSampler

def param_shift_gradient(theta_vector, ansatz_isa_list, basis_list, shots=shots):
    #sampler = qiskitSampler(options={"shots": shots, "seed": rng.integers(1000000)})
    sampler = qiskitSampler(options={"shots": shots})
    param_shift_sampler_gradient = ParamShiftSamplerGradient(sampler)

    prob_gradients = param_shift_sampler_gradient.run(ansatz_isa_list, [theta_vector for _ in ansatz_isa_list], None).result().gradients

    results_dic = {basis:counts for (basis, counts) in zip(basis_list, prob_gradients)}
    return results_dic

In [ ]:
# computes the gradient of the symmetrized kl loss on the theta parameters
def KL_div_symm_gradient(gradients, true_data, test_data, num_parameters, epsilon_loss = epsilon_loss):
    kl_gradients = np.zeros(num_parameters)

    for parameter_index in range(num_parameters):
        grad_true_test = 0
        grad_test_true = 0

        # here, let me assume true data and test data can have different number of shots
        # but the shot in each basis should be the same
        true_data_total_shots = sum(list(true_data.values())[0].values())
        test_data_total_shots = sum(list(test_data.values())[0].values())

        for basis in true_data: # just a note that if there are hallucinated measurements in 
            # bases/states not included in the true data then the kl divergence will not account for these (at least as presented in the paper)
            for state in true_data[basis]:
                true_counts = true_data[basis][state]
                test_counts = test_data[basis].get(state, 0)
                true_prob   = true_counts / true_data_total_shots
                test_prob   = test_counts / test_data_total_shots

                gradient_test = gradients[basis][parameter_index].get(int(state, 2), 0)
                grad_true_test -= gradient_test * ((true_prob) / (test_prob +epsilon_loss))     
                
        for basis in test_data: 
            for state in test_data[basis]:
                true_counts = true_data[basis].get(state, 0)
                test_counts = test_data[basis][state]
                true_prob   = true_counts / true_data_total_shots
                test_prob   = test_counts / test_data_total_shots
                
                gradient_test = gradients[basis][parameter_index].get(int(state, 2), 0)          
                grad_test_true += gradient_test * (1 + np.log((test_prob) / (true_prob +epsilon_loss)))

        grad_true_test /= len(true_data)
        grad_test_true /= len(test_data)

        kl_gradients[parameter_index] = grad_true_test + grad_test_true

    return kl_gradients

In [ ]:
from qiskit_algorithms.optimizers import OptimizerResult

# I also added an extra convergence checker for adam as well
def param_shift_gradient_descent_adam(maxiter, learning_rate, decay_rate1, decay_rate2, epsilon, initial_theta, basis_list, true_data, shots, print_message=True, inst_stop_loss = 0.08, conv_threshold=0):
    result = OptimizerResult()
    result.nfev = 0
    cur_theta = initial_theta
    past_loss = 100000000000
    norm_adjusted_gradient = 100000000000

    E_g2 = np.zeros(initial_theta.shape)
    E_g = np.zeros(initial_theta.shape)

    for iter in range(maxiter):
        (cur_loss, test_data) = mod_compute_kl_loss(cur_theta, ansatz_isa_list, basis_list, GHZ_measurement, shots, param_vec, print_message=False)
        result.nfev += 1

        if print_message:
            print(f"ITERATION: {iter+1}; loss at start of iteration is {cur_loss} \t Relative change in loss {(cur_loss-past_loss)/past_loss}")
        
        # When resullt is good enough we can exit early in order to save time
        # if we found a very low loss we just break
        if (cur_loss < inst_stop_loss):
            print(f"Found Small Loss EARLY at start of iter: {iter+1}")
            break
        # handles convergence by checking if gradient was small enough
        if (norm_adjusted_gradient < conv_threshold and (cur_loss-past_loss) < 0):
            print(f"Found Convergence EARLY at start of iter: {iter + 1}")
            break
        
        # calcualting kl gradient using parameter shift rule
        circuit_gradients = param_shift_gradient(cur_theta, ansatz_isa_list, basis_list, shots=shots)
        result.nfev += 2 * len(cur_theta)
        kl_gradient = KL_div_symm_gradient(circuit_gradients, true_data, test_data, len(cur_theta))
        
        # applying adam propogation
        E_g = decay_rate1 * E_g + (1-decay_rate1) * kl_gradient
        E_g2 = decay_rate2 * E_g2 + (1-decay_rate2) * np.square(kl_gradient)

        bias_cor_E_g = E_g / (1 - decay_rate1 ** (iter+1))
        bias_cor_E_g2 = E_g2 / (1 - decay_rate2 ** (iter+1))
        
        adjusted_gradient = bias_cor_E_g / (np.sqrt(bias_cor_E_g2) + epsilon)
        cur_theta = cur_theta - learning_rate * adjusted_gradient
        
        # saves the norm adjusted gradient  
        norm_adjusted_gradient = np.linalg.norm(adjusted_gradient)
        
        if(print_message):
            print(f"Norm of Adam adjusted gradient: {norm_adjusted_gradient}")
            
        past_loss = cur_loss



    (loss, _) = mod_compute_kl_loss(cur_theta, ansatz_isa_list, basis_list, GHZ_measurement, shots, param_vec, print_message=print_message)
    result.nfev += 1
    
    result.fun = loss
    result.jac = kl_gradient
    result.x = cur_theta
    return result

def finite_difference_forward_adam(loss_func, maxiter, learning_rate, decay_rate1, decay_rate2, epsilon, initial_theta, pertebation=0.1, print_message=True, inst_stop_loss = 0.09, conv_threshold=0.35):
    cur_theta = initial_theta
    past_loss = 100000000000
    norm_adjusted_gradient = 100000000000

    h = np.zeros(initial_theta.shape)
    fd_gradient = np.zeros(initial_theta.shape)
    E_g2 = np.zeros(initial_theta.shape)
    E_g = np.zeros(initial_theta.shape)
    
    for iter in range(maxiter):
        # calculating gradient according to finite difference method
        cur_loss = loss_func(cur_theta)
        if print_message:
            print(f"ITERATION: {iter+1}; loss at start of iteration is {cur_loss}  \t Relative change in loss {(cur_loss-past_loss)/past_loss}")
        
        # When resullt is good enough we can exit early in order to save time
        # if we found a very low loss we just break
        if (cur_loss < inst_stop_loss):
            print(f"Found Small Loss EARLY at start of iter: {iter+1}")
            break
        # handles convergence by checking if gradient was small enough
        if (norm_adjusted_gradient < conv_threshold and (cur_loss-past_loss) < 0):
            print(f"Found Convergence EARLY at start of iter: {iter + 1}")
            break
        
        # calculating approximate gradient with forward finite difference
        for i, _ in enumerate(cur_theta):  
            h[i] = pertebation
            fd_gradient[i] = (loss_func(cur_theta + h) - cur_loss) / h[i]
            h[i] = 0

        # applying adam propogation
        E_g = decay_rate1 * E_g + (1-decay_rate1) * fd_gradient
        E_g2 = decay_rate2 * E_g2 + (1-decay_rate2) * np.square(fd_gradient)

        bias_cor_E_g = E_g / (1 - decay_rate1 ** (iter+1))
        bias_cor_E_g2 = E_g2 / (1 - decay_rate2 ** (iter+1))

        adjusted_gradient = bias_cor_E_g / (np.sqrt(bias_cor_E_g2) + epsilon)
        cur_theta = cur_theta - learning_rate * adjusted_gradient
        
        # saves the norm adjusted gradient  
        norm_adjusted_gradient = np.linalg.norm(adjusted_gradient)
        
        if(print_message):
            print(f"Norm of Adam adjusted gradient: {norm_adjusted_gradient}")
            
        past_loss = cur_loss



    loss = loss_func(cur_theta)
    
    result = OptimizerResult()
    result.fun = loss
    result.jac = fd_gradient
    result.x = cur_theta
    return result

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import mplhep as hep

hep.style.use(hep.style.CMS)

def plot_hist_optimizer(optimizer_record, optimizer, topic_str, xlabel, save_location, showTitle=False):
    plt.hist(optimizer_record)
    plt.xlabel(xlabel)
    plt.ylabel("Frequency")
    if (showTitle):
        plt.title(topic_str + f" - Alternating Ansatz + KL divergence + {optimizer}")
    plt.savefig(save_location, bbox_inches='tight')
    plt.clf()

def plot_all_histogram(record, optimizers, topic_str, xlabel, save_location, showTitle=False):
    flat_record = [i for list in record for i in list]   
    bins = np.histogram(flat_record, bins=25)[1]

    plt.hist(record, bins=bins, alpha = 0.7, label = optimizers)


    plt.xlabel(xlabel)
    plt.ylabel("Frequecy")
    plt.legend()

    if(showTitle):
        plt.title("Main " + topic_str + f" - Alternating Ansatz + KL divergence")

    plt.savefig(save_location, bbox_inches='tight')
    plt.clf()

def plot_strip_plot(record, optimizers, topic_str, ylabel, save_location, includeBox=True, logScale=False, showTitle=False, invertYAxis=False):
    if (invertYAxis):
        plt.gca().invert_yaxis()
    
    reformatted_record = [(element, optimizers[i]) for i, optimizer_record in enumerate(record) for element in optimizer_record]
    df = pd.DataFrame(reformatted_record, columns=[ylabel, "Optimizers"])
    
    ax = sns.stripplot(data=df, x="Optimizers", y=ylabel, hue="Optimizers", legend=False)

    if (includeBox):
        sns.boxplot(data=df, x="Optimizers", y=ylabel, showcaps=False, boxprops={'facecolor':'None'},
                    showfliers=False,whiskerprops={'linewidth':0})
    
    if (logScale):
        plt.yscale('log')

    if(showTitle):
        plt.title("Main " + topic_str + f" Strip Plot - Alternating Ansatz + KL divergence")
        
    plt.xticks(rotation = 90)
    plt.savefig(save_location, bbox_inches='tight')
    plt.clf()



In [ ]:
# code for testing all optimizers
from qiskit_algorithms.optimizers import SPSA
from qiskit_algorithms.optimizers import GradientDescent
from qiskit_algorithms import utils

import scipy.optimize as opt
import time
import warnings
from wakepy import keep
from pybads import BADS

reruns = 25
folder = "saved_final_no_seed_spin_chain_state"
# first 0-6 do reasonably well, 7-10 have poorer performance
optimizers = ["Powell", "PYBADS", "Parameter_Shift", "Finite_Difference", "COBYLA", "SPSA", "SPSA_Optimized", "Nelder-Mead", "L-BFGS-B", "COBYQA", "SLSQP"]
warnings.filterwarnings("ignore", category=DeprecationWarning)

function_call_records = [[-1 for _ in range(reruns)] for _ in range(len(optimizers))]
fidelities_record = [[-1 for _ in range(reruns)] for _ in range(len(optimizers))]
elapsed_cpu = [[-1 for _ in range(reruns)] for _ in range(len(optimizers))]
elapsed_wall = [[-1 for _ in range(reruns)] for _ in range(len(optimizers))]

# wakepy can keep computer running program and prevents screen from going to sleep
with keep.presenting():
    for j, optimizer in enumerate(optimizers):
        loss_value_records = [0] * reruns
        
        total_elapsed_cpu = 0
        total_elapsed_wall = 0

        for i in range(0, reruns ):
            start_cpu = time.process_time()
            start_wall = time.perf_counter()

            global loss_values
            global thetas

            loss_values = []
            thetas = []

            theta = initialize_theta_random(circ_depth=depth, num_qbits = n_qubits)
            theta_vector = np.reshape(theta, theta.size)

            # handles seperate case of PYBADS 
            if (optimizer == "PYBADS"):
                # establish bounds 
                lb = np.full(theta.size, -10.0)
                ub = np.full(theta.size, 10.0)

                plb = np.full(theta.size, -2.0*pi)
                pub = np.full(theta.size, 2.0*pi)

                # try to find the minimal
                bads = BADS(compute_kl_loss_filled_quiet, theta_vector, lb, ub, plb, pub)
                results = bads.optimize()
            # handles case of SPSA
            elif (optimizer == "SPSA"):
                spsa = SPSA(maxiter=300)
                results = spsa.minimize(compute_kl_loss_filled, theta_vector)

            # handles SPSA with hyperparameters adjusted with random search optimization 
            elif (optimizer == "SPSA_Optimized"):
                spsa = SPSA(maxiter=300)

                # coefficients for SPSA (these are just the defaults)
                c=0.1258
                A=0.3186
                a1=0.4739
                alpha=0.6374
                gamma=0.06059

                (lr, p) = spsa.calibrate(compute_kl_loss_filled, theta_vector, c=c, stability_constant = A, target_magnitude=a1, alpha=alpha, gamma=gamma)
                spsa.learning_rate = lr
                spsa.perturbation = p
                
                results = spsa.minimize(compute_kl_loss_filled, theta_vector)
            # handles case of finite difference gradient descent (forward approach)
            elif (optimizer == "Finite_Difference"): 
                maxiter = 300
                l_rate = 0.05
                d_rate1 = 0.9
                d_rate2 = 0.999
                pert = 0.08
                inst_stop_loss = 0.08
                conv = 0.3

                results = finite_difference_forward_adam(compute_kl_loss_filled_quiet, maxiter, l_rate, d_rate1, d_rate2, epsilon_loss, theta_vector, pertebation=pert, inst_stop_loss=inst_stop_loss, conv_threshold=conv)

            elif (optimizer == "Parameter_Shift"):
                maxiter = 300
                l_rate = 0.02
                d_rate1 = 0.9
                d_rate2 = 0.999
                conv = 0.2
                loss = 0.08

                results = param_shift_gradient_descent_adam(maxiter, l_rate, d_rate1, d_rate2, epsilon_loss, theta_vector, basis_list, GHZ_measurement, shots, inst_stop_loss=loss, conv_threshold=conv)
            else:
                # establish bounds
                bounds = [(-2.0*pi, 2.0*pi) for i in range(0, theta.size)]
                args = (ansatz_isa_list, basis_list, GHZ_measurement, shots, param_vec)

                # try to find the minimal
                results = opt.minimize(compute_kl_loss, theta_vector, args = args, method = optimizer, bounds=bounds, options={"maxiter":max_iter})
            
            
            loss_value_records[i] = loss_values

            end_cpu = time.process_time()
            end_wall = time.perf_counter()

            elapsed_cpu[j][i] = end_cpu - start_cpu
            elapsed_wall[j][i] = end_wall - start_wall
            total_elapsed_cpu += elapsed_cpu[j][i]
            total_elapsed_wall += elapsed_wall[j][i]

            ansatz_state = Statevector.from_instruction(ansatz.assign_parameters({param_vec : results.x}))
            fidelities_record[j][i] = qiskit.quantum_info.state_fidelity(ghz_state, ansatz_state)

            if (optimizer[:15] != "Parameter_Shift"):
                function_call_records[j][i] = len(loss_value_records[i])
            else:
                # parameter shift function calls is a better analogue to the number loss_values found in other optimizers
                # since parameter shift only calls the loss function once per iteration
                # However, the parameter shift itself runs the circuit some times as well. 
                function_call_records[j][i] = results.nfev
                np.save(f"{folder}/{optimizer}/num_function_calls_iter{i}.npy", function_call_records[j][i])

            # save all of the results as npy
            np.save(f"{folder}/{optimizer}/params_training_iter{i}.npy", thetas)
            np.save(f"{folder}/{optimizer}/loss_training_iter{i}.npy", loss_values)
            np.save(f"{folder}/{optimizer}/final_theta_iter{i}.npy", results.x)
            np.save(f"{folder}/{optimizer}/fidelity_iter{i}.npy", fidelities_record[j][i])
            np.save(f"{folder}/{optimizer}/elapsed_cpu_times_iter{i}.npy", elapsed_cpu[j][i])
            np.save(f"{folder}/{optimizer}/elapsed_wall_times_iter{i}.npy", elapsed_wall[j][i])


        # makes a summary result txt file for readability
        with open(f"{folder}/all_optimizers_summary.txt", 'a') as file:
            file.write(f"[OPTIMIZER]: {optimizer}\n")

            file.write(f"AVG Wall time elapsed: {total_elapsed_wall/reruns}\n")
            file.write(f"ALL Wall times elapsed: {elapsed_wall[j]}\n")
            file.write(f"AVG CPU time elapsed: {total_elapsed_cpu/reruns}\n")
            file.write(f"ALL CPU times elapsed: {elapsed_cpu[j]}\n")
            
            file.write(f"Function Calls: {function_call_records[j]}\n")

            file.write(f"Fidelities: {fidelities_record[j]}\n\n")

        # plot the loss function and save the plots
        for i in range(0, reruns):
            plt.plot(loss_value_records[i], label=f"KL Loss " + str(i), alpha=0.5)

        # plot the loss function over the total number of loss function calls and save plots
        plt.ylim(0, 3)
        plt.xlabel("Iteration")
        plt.ylabel("Loss")
        plt.title(f"Loss - Alternating Ansatz + KL divergence + {optimizer}")
        plt.savefig(f"{folder}/{optimizer}/plots_loss_func.png", bbox_inches='tight')
        plt.clf()

        # plot the number of function calls in a histogram
        plot_hist_optimizer(function_call_records[j], optimizer, "Func Call", "Function calls", f"{folder}/{optimizer}/plots_func_call.png")

        # plot the wall times in a histogram
        plot_hist_optimizer(elapsed_wall[j], optimizer, "Wall Time", "Wall Time (sec)", f"{folder}/{optimizer}/plots_wall_time.png")

        # plot the cpu times in a histogram
        plot_hist_optimizer(elapsed_cpu[j], optimizer, "CPU Time", "CPU Time (sec)", f"{folder}/{optimizer}/plots_cpu_time.png")

        # plot the fidelity as a histogram with horizontal axis being fidelity and verticle axis is frequency and save plots.
        plot_hist_optimizer(fidelities_record[j], optimizer, "Fidelity Histogram", "Fidelity", f"{folder}/{optimizer}/plots_fidelity_hist.png")

        # plot the fidelity as a bar graph (verticle axis is fidelity and horizontal is trial number) and save plots
        plt.bar([f"Trial {i}" for i in range(0, reruns)], fidelities_record[j])
        plt.xticks(rotation = 45)
        plt.ylabel("Fidelity")
        plt.title(f"Fidelity Bar Graph - Alternating Ansatz + KL divergence + {optimizer}")

        plt.savefig(f"{folder}/{optimizer}/plots_fidelity_bar.png", bbox_inches='tight')
        plt.clf()

